In [ ]:
'''
Test - 2.7 Spurious in band carrier 
Test description- This test verifies, there is no spurious present in-band (between start and stop frequency ie span ) of the carrier above the defined error limit
Equipment required - 
Lucid device - LS1291D
Spectrum analyzer- Agilent Technologies, E4440A (PSA Series spectrum analyzer)
what is happening in thi script ?
 - After initializing the device, all the parameter are define in 2nd section,
 - start and stop frequency is selected +/-500  to the signal frequency
 - using 3 marker, the script is checking the peak, peak right and peak left, 
 - if all these marker values are same, then we conclude that there is not spurious above -60dBc in band to the carrier.
 -and if the marker value is not then we check if the power level of each marker to be above threshold, then we conclude the final results
-  A test report will be generated at the end of the script and is saved as a text file in the same directory '''''
import numpy as np
import pyvisa as visa
import socket
import time
import re  # Set the IP Address and port

handle = 'TCPIP0::192.168.7.1::5025::SOCKET'


def send_scpi_cmd(cmd, handle):
    try:
        resourceManager = visa.ResourceManager()
        session = resourceManager.open_resource(handle)

        # Need to define the termination string
        session.write_termination = '\n'
        session.read_termination = '\n'

        session.write(cmd)
        session.close()
    except Exception as e:
        print('[!] Exception: ' + str(e))


def send_scpi_query(query, handle):
    try:
        resourceManager = visa.ResourceManager()
        session = resourceManager.open_resource(handle)

        # Need to define the termination string
        session.write_termination = '\n'
        session.read_termination = '\n'

        response = str(session.query(query))
        session.close()
        return response

    except Exception as e:
        print('[!] Exception: ' + str(e))


send_scpi_cmd('*CLS', handle)
query = '*IDN?'
temp = send_scpi_query(query, handle)
print("Connected to :", temp)
def connect_spectrum_via_lan():
    device_address = 'TCPIP::192.90.70.36::5025::SOCKET'
    try:
        rm = visa.ResourceManager()
        spectrum_analyzer = rm.open_resource(device_address)
        spectrum_analyzer.timeout = 2000
        spectrum_analyzer.write_termination = '\n'
        spectrum_analyzer.read_termination = '\n'
        print(spectrum_analyzer)
    except visa.Error as e:
        print("Error while connecting: {}".format(e))
    try:
        # Query the *IDN? command to get the identification string
        spectrum_analyzer.write('*IDN?')
        # spectrum_analyzer.timeout = 1000
        idn_response = spectrum_analyzer.read()
        spectrum_analyzer.write('*RST')
        spectrum_analyzer.write('*CLS')

        print("IDN Response: {}".format(idn_response))
    except visa.Error as e:
        print("Error reading IDN: {}".format(e))
    return spectrum_analyzer

def power_at_centerFrequency(spectrum_analyzer):
    spectrum_analyzer.write(':CALC:MARK1:STAT ON;:CALC:MARK1:MAX')
    time.sleep(1)
    spectrum_analyzer.write('CALC:MARK1:X?')
    time.sleep(1)
    resp = spectrum_analyzer.read()
    time.sleep(1)
    freq_out = float(resp) / 1e6
    #print('centre frequency = {} MHz'.format(freq_out))
    time.sleep(1)
    spectrum_analyzer.write('CALC:MARK1:Y?')
    time.sleep(1)
    power_max = spectrum_analyzer.read()
    time.sleep(1)
    print('power ={1} dBm at frequency = {0} MHz'.format(freq_out, power_max))
    return power_max,freq_out
def get_right_peak(freq_out):
    freq_right = 0
    while float(freq_right)< (float(freq_out)+100):
        spectrum_analyzer.write(':CALCulate:MARKer1:MAXimum:RIGHt')
        spectrum_analyzer.write('CALC:MARK1:X?')
        time.sleep(1)
        # spectrum_analyzer.write(':CALC:FUNC SPU')
        # spectrum_analyzer.write(':INIT:IMM')
        # time.sleep(10)
        # spectrum_analyzer.write(':FETC:SPU:MAX?')
        resp = spectrum_analyzer.read()
        # print(resp)
        freq_right = float(resp) / 1e6

    spectrum_analyzer.write('CALC:MARK1:Y?')
    time.sleep(1)
    pow_marker = spectrum_analyzer.read()
    print('Right peak at {0}MHz power ={1} dBm'.format(freq_right, pow_marker))
    return freq_right, pow_marker
def get_left_peak(freq_out):
    freq_left = 0
    while float(freq_left)< (float(freq_out)-100):
        spectrum_analyzer.write(':CALCulate:MARKer1:MAXimum:LEFt')
        spectrum_analyzer.write('CALC:MARK1:X?')
        time.sleep(1)
        resp = spectrum_analyzer.read()
        freq_left = float(resp) / 1e6
    spectrum_analyzer.write('CALC:MARK1:Y?')
    time.sleep(1)
    pow_marker= spectrum_analyzer.read()
    print('Left peak at {0}MHz power ={1} dBm'.format(freq_right, pow_marker))
    return freq_left,pow_marker
# Initilization
#send_scpi_cmd('*RST', handle)
resp = send_scpi_query('*IDN?', handle)
print("Connected to :",resp)
#send_scpi_cmd(':SYST:ERR?', handle)
# send_scpi_cmd(':INST 1;:INST:ACT 1', handle)

test_success = True
####################################################################
# Parameters
frequency_list =[1000,2000,3500,4000,550,2300,1500,3500,4000,3500,4000,550,2300,1500,3500,4000]
start_freq_tb = [500,1500,2400,2400,3400,3900,4900,5900,5900,6900,6900,7900,8900,9900,10900,10900]
stop_freq_tb = [1500,2400,3400,3400,3900,4900,5900,6900,6900,7900,7900,8900,9900,10900,11900,11900]
output = np.zeros(len(frequency_list)) # To store the output frequency from spectrum analyzer (optional step)
power_in_dBm = 5# power level required for the test in dBm
ref_level = power_in_dBm+5 # reference level on the spectrum must be 5dBm above the signal power level
error_limit = -90 # error limit in dBm
threshold = error_limit - power_in_dBm
## Test report parameters
################
# Parameters for test report
table_i=0
serial_no = np.zeros(len(frequency_list))
input_list_tb = np.zeros(len(frequency_list))

output_list_tb= np.zeros(len(frequency_list))
output_status_tb= []# To store the output frequency from spectrum analyzer (optional step)
error_limit = -60+np.zeros(len(frequency_list)) # error limit in dBm
res_BW = 30 # resolution bandwidth in KHz
spectrum_analyzer = connect_spectrum_via_lan()
spectrum_analyzer.write(':INIT:REST')
print('Initialized')
spectrum_analyzer.write(':BWID:RES {}KHz'.format(res_BW))  # resolutrion bandwidth
for table_i in range(len(frequency_list)):  # iterating over each frequency (main loop)
    center_frequency = frequency_list[table_i]
    serial_no[table_i] = table_i + 1
    input_list_tb[table_i] = center_frequency
    send_scpi_cmd(':FREQuency {}e6'.format(center_frequency), handle)
    resp = send_scpi_query(':FREQuency?', handle)
    print("Frequency :", resp)
    send_scpi_cmd(':POWer {0}'.format(power_in_dBm), handle)
    resp = send_scpi_query(':POWer?', handle)
    print("Power :", resp)
    send_scpi_cmd(':OUTPut ON', handle)
    spectrum_analyzer.write('DISP:WIND:TRAC:Y:RLEV 10 dbm')
    # resolution bw = 30 k
    spectrum_analyzer.write(':FREQ:STAR {}MHz'.format(start_freq_tb[table_i]))
    time.sleep(2)
    spectrum_analyzer.write(':FREQ:STOP {}MHz'.format(stop_freq_tb[table_i]))
    time.sleep(2)
    spectrum_analyzer.write(':CALC:MARK:PEAK:THR {}'.format(threshold))
    time.sleep(1)
    peak_max, freq_out = power_at_centerFrequency(spectrum_analyzer)
    #     print(peak_max,freq_out)

    #send_scpi_cmd(':OUTPut OFF', handle)

    spectrum_analyzer.write(':CALC:MARK:PEAK:THR {0}'.format(threshold))
    time.sleep(1)
    spectrum_analyzer.write(':CALCulate:MARKer1:PEAK:EXCursion 0')
    time.sleep(1)
    #spectrum_analyzer.write(':CALC:MARK1:STAT ON;:CALC:MARK1:MAX')



    freq_right, pow_m2 =get_right_peak(freq_out)
    freq_left, pow_m3 = get_left_peak(freq_out)
    time.sleep(1)
    # #spectrum_analyzer.write(':CALC:MARK1:STAT ON;:CALC:MARK1:MAX')
    # spectrum_analyzer.write(':CALCulate:MARKer1:MAXimum:LEFT')
    # spectrum_analyzer.write('CALC:MARK1:X?')
    # time.sleep(1)
    # # spectrum_analyzer.write(':CALC:FUNC SPU')
    # # spectrum_analyzer.write(':INIT:IMM')
    # # time.sleep(10)
    # # spectrum_analyzer.write(':FETC:SPU:MAX?')
    # resp = spectrum_analyzer.read()
    # # print(resp)
    # peak_left = float(resp) / 1e6
    # spectrum_analyzer.write('CALC:MARK1:Y?')
    # time.sleep(1)
    # pow_m3 = spectrum_analyzer.read()
    # print('Left peak at {0}MHz power ={1} dBm'.format(peak_left, pow_m3))

    # if peak_max == freq_right == freq_left:
    #     output_status_tb.append("pass")
    #     print("Test pass")
    output_list_tb[table_i] = max(pow_m2,pow_m3)
    if float(pow_m2) <= threshold and float(pow_m3) <= threshold:
        output_status_tb.append("pass")
        print("Test pass")
    else:
        output_status_tb.append("FAIL")
        test_success = False
        print("Test Fail")
print("test completed")
# disconnecting the instruments
spectrum_analyzer.close()
##########################################
#Test report
from tabulate import tabulate
# Sample data for the table
data = list(zip(serial_no,input_list_tb,start_freq_tb,stop_freq_tb,error_limit,output_list_tb,output_status_tb))
# Creating the table
table = tabulate(data, headers=["Test step","Output frequency","Start frequency","Stop frequency","Error Limits (in dBc)","Max Spurious","PASS/FAIL"], tablefmt="grid")
# Displaying the table
print(table)
# Writing the table to a text file
with open("Spurious in-band test report.txt", "w") as f:
    f.write(table)